<a href="https://colab.research.google.com/github/omkarade/Speaker-Recognition-using-Transfer-Learning-And-One-Shot-Learning/blob/main/model_training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
import os
import numpy as np
import pandas as pd
from keras.utils import np_utils
from keras.layers import Dense, Dropout, Activation, Flatten
from tensorflow import keras
from keras import optimizers
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier
import tensorflow as tf
from sklearn import svm
from keras.models import Model
import warnings
from numpy import loadtxt
from xgboost import XGBClassifier
warnings.filterwarnings("ignore")


In [ ]:
train=os.listdir('train')
y_train=np.array('y_train')

**loading rgb Spectrogram into [513,800,3] array size**

In [ ]:
Xtrain=[]
for i in train:   
    image=tf.keras.preprocessing.image.load_img('train/{}'.format(i), color_mode='rgb', target_size= (513, 800, 3))
    image=np.array(image)
    Xtrain.append(image)
Xtrain=np.array(Xtrain)
y_train=np.array(y_train)    

**loading VGG-16 model**

In [ ]:
from keras.applications import VGG16
vgg_model = VGG16(weights='imagenet',include_top=False, input_shape=(513, 800, 3))

In [ ]:
for i,layer in enumerate(vgg_model.layers):
  layer.trainable = False


**Adding Flatten layer**

In [ ]:
x = vgg_model.output
x = Flatten()(x) # Flatten dimensions to for use in FC layers
transfer_model = Model(inputs=vgg_model.input, outputs=x)


In [ ]:
model2 = Model(inputs=transfer_model.input, outputs=transfer_model.get_layer('flatten').output)


**Passing [513,800,3]dimension Spectrogram into pre-trained model to get 204800-dimension representation of 5 sec wav file**

In [ ]:
svm_x_train = []
svm_y_train = []
for i in range(len(Xtrain)):
  x_1 = np.expand_dims(Xtrain[i], axis=0)
  flatten_2_features = model2.predict(x_1)
  svm_x_train.append(flatten_2_features)
  svm_y_train.append(y_train[i])


In [6]:
svm_x_test=np.load('test.npy')
svm_y_test=np.load('ytest.npy')

**training SVM model**

In [5]:
svm_x_train = np.array(svm_x_train)
clf = svm.SVC(kernel='rbf', class_weight='balanced')
dataset_size = len(svm_x_train) 
svm_x_train = np.array(svm_x_train).reshape(dataset_size,-1)
svm_y_train = np.array(svm_y_train)
clf.fit(svm_x_train, svm_y_train)


SVC(class_weight='balanced')

* trump_mimic# -------------- 10
* original _Trump# ---------- 9
* male_Kannada# ------------- 8
* female_marathi# ----------- 7
* female_english_1#---------- 6
* english_male_2#------------ 5
* Shyam_Rangeela# ----------- 4
* PMmodi#-------------------- 3
* Malayalam_man#------------- 2
* Jayanti_khatri#------------ 1


In [11]:
print(classification_report(clf.predict(svm_x_test),svm_y_test))

              precision    recall  f1-score   support

           0       1.00      0.50      0.67        12
           1       0.83      1.00      0.91         5
           2       1.00      0.80      0.89         5
           3       1.00      0.95      0.98        22
           4       0.79      1.00      0.88        11
           5       1.00      1.00      1.00         4
           6       0.00      0.00      0.00         0
           7       1.00      1.00      1.00         3
           8       1.00      0.75      0.86         4
           9       1.00      0.75      0.86         8
          10       0.80      1.00      0.89         4

    accuracy                           0.86        78
   macro avg       0.86      0.80      0.81        78
weighted avg       0.95      0.86      0.88        78

